In [8]:
# Load a list of integrals of Laguerre Gaussians
Lag=load('laguerre_integrals')

##########################################################################################################
# -----------------------------------------------------------------
#  nev_verify
# -----------------------------------------------------------------
#
# INPUT:
# - genus: the genus for which we want to prove a bound
# - syst:  the systole at which we want to prove a bound
# - param: a pair (Z,W), where:
#          * Z is a list of prescribed double roots for f(sqrt(x))
#          * W is a list of prescribed double roots for f^(sqrt(x))
#
# THE PROGRAM:
# - Computes a Fourier pair (f,f^) based on "syst" and "param":
#    f(x) will be of the form u(x^2)*e^(-x^2/2) for some polynomial u to be determined
#    f^(y) is v(y^2)*e^(-y^2/2) for some polynomial v to be determined
#    the degree d of u and v are determined by the total number of parameters: it's the minimal degree such
#    that there is a pair of non-zero polynomials with the prescribed values. This implies that the pair
#    (u,v) is uniquely determined by "syst" and "param"    
# - Verifies whether the pair (f,f^) satisfies the conditions of Theorem 10.1 for L = 1/4
# - If so, gives a rigorous estimate for the bound from that Theorem
#
# OUTPUT:
# - If the conditions are satisfied: the computed bound
# - If not, an error message
##########################################################################################################

def nev_verify(genus,syst,param):
    print('\n\n----------------------------------------------------------------')
    print('nev_verify')
    print('systole:  ', syst, '\n\n')
    
    (Z,W) = param
    
    # Make sure the entries of Z and W and the systole are treated as rational numbers
    Z = [QQ(elem) for elem in Z]
    W = [QQ(elem) for elem in W]
    S = QQ(syst^2)
    
    m = len(Z) #number of forced double roots for f
    n = len(W) #number of forced double zeros for f^
    
    # Compute the polynomials 
    
    (u,v) = nev_poly(syst,param)

    print('Computed the polynomials (u,v) based on the prescribed zeroes')
    
    u_der = u.derivative()
    v_der = v.derivative()

    u_der_der = u_der.derivative()
    v_der_der = v_der.derivative()

    # Check part of the sign conditions from []    
    if u(S)>0:
        return 'f is positive at the systole'
    
    if sgn(u.leading_coefficient())==1:     #check that u is eventually negative
        return 'f takes positive values near infinity'

    if sgn(v.leading_coefficient())==-1:  #check that v is eventually positive        
        return 'f^ takes negative values near infinity'  
    
    print('Checked the first sign conditions on f and f^')
            
    
    # We check that f^(sqrt(x)) = v(x)*e^(-x/2) is unimodal on -1/4 to 0
    # note that the derivative of f^(sqrt(x)) with respect to x is:
    # (2*v'(x)-v(x))*exp(-x/2)/2
    # we are thus asking f^(sqrt(x))' to have at most one zero, which needs to correspond to a maximum
    
    # Count extrema    
    extrema = pari.polsturm(2*v_der-v,[-1/4,0])
    if extrema>1:
        return 'f^ has more than one local extremum on the imaginary segment'
            
    if extrema==1:
        # Check that the extremum is a local maximum
        if 2*v_der(-1/4)-v(-1/4)<=0:
            return 'f^(sqrt(x)) is not increasing at -1/4'
            
        if 2*v_der(0)-v(0)>=0:
            return 'f^(sqrt(x)) is not decreasing at 0'
            
    M = min(RBF(v(-1/4)*exp(1/8)).upper(),RBF(v(0)).upper())
        
    if M<=0: 
        return 'f^ takes negative values on the imaginary segment'
        
    # Compute a bound on N_small
    val = 1+(2*(genus-1)-RDF(v(-1/4)*exp(1/8)))/M
    
    print('Verified that f^(r(lambda)) is unimodal on the interval [0,1/4] and determined its minimum')

    if pari.polsturm(u,[S,Infinity])>m+1:
        return 'f has an extra zero'
                
    if pari.polsturm(v,[-1/4,Infinity])>n:
        return 'f^ has an extra zero'

    # Check that u and v vanish to the right degree at the prescribed roots
    for j in range(m):
        if u(Z[j])!=0 :
            return 'f does not vanish at one of the prescribed zeros'
        if u_der(Z[j])!=0 :
            return 'One of the prescribed zeros of f is not double'
        if u_der_der(Z[j])>=0 :
            return 'The second derivative of f at one of the double roots is non-negative'
    for j in range(n):                
        if v(W[j])!=0 :
            return 'f^ does not vanish at one of the prescribed zeros'
        if v_der(W[j])!=0 :          
            return 'One of the prescribed zeros of f^ is not double'
        if v_der_der(W[j])<=0 :        
            return 'The second derivative of f^ at one of the double roots is non-positive'

    print('Verified that the roots of f and f^ are exactly as prescribed')

        
    return val 

##########################################################################################################
# -----------------------------------------------------------------
#  nev_poly
# -----------------------------------------------------------------
# INPUT:
# - syst: the systole at which we want to prove a bound
# - param: a pair (Z,W), where:
#          * Z is a set of prescribed double roots for f
#          * W is a set of prescribed double roots for f^
# 
# THE PROGRAM:
# - Computes a pair (u,v) of polynomials based on "syst" and "param":
#    f(x) will be of the form u(x^2)*e^(-x^2/2) for some polynomial u to be determined
#    f^(y) is v(y^2)*e^(-y^2/2) for some polynomial v to be determined
#    the degree d of u and v are determined by the total number of roots: it's the minimal degree such
#    that there is a pair of non-zero polynomials with the prescribed values. This implies that the pair
#    (u,v) is uniquely determined by "syst" and "param"    
#
# OUTPUT:
# (u,v)
##########################################################################################################


def nev_poly(syst,param):
    
    (Z,W) = param

    # Make sure the entries of Z and W and the systole are treated as rational numbers
    Z = [QQ(elem) for elem in Z]
    W = [QQ(elem) for elem in W]
    S = QQ(syst^2)
    
    m = len(Z) #number of forced double roots for f
    n = len(W) #number of forced double zeros for f^
    c = 2+2*(m+n) #number of conditions imposed
    
    # Compile a list of Laguerre polynomials and their derivatices
    P.<x> = PolynomialRing(QQ) 
    L = [ P(gen_laguerre(n,-1/2,x)) for n in range(c)]
    L_der = [ L[n].derivative() for n in range(c)]
    
    # Set up and solve the linear algebra problem
    
    A = [[L[j](S) for j in range(c)]]
    
    B = []
    for point in Z:
        B.append([L[j](point) for j in range(c)])
        B.append([L_der[j](point) for j in range(c)])
        
    
    C = []
    for point in W:
        C.append([(-1)^j*L[j](point) for j in range(c)])
        C.append([(-1)^j*L_der[j](point) for j in range(c)])  
        
    # Load the integrals of the Laguerre Gaussians that we computed beforehand.
    # The correctness of these integrals does not need to be verifiable, they are only used to set linear
    # condition on v and don't influence whether or not u(x^2)*exp(-x^2/2) and v(y^2)*exp(-y^2)/2
    # are each other's Fourier transforms.
    # The fact that the pair (u,v) satisfies all the necessary conditions is verified a posteriori in
    # nev_verify.        
    D = [[(-1)^j*Lag[j] for j in range(c)]]
    
    M = pari(matrix(QQ,A+B+C+D)) #matrix of values of the Laguerre polynomials and their derivative or combinations thereof
    Y = pari(matrix(QQ,[0]+[0]*(c-2)+[1]).transpose()) #this is the vector of target values
    X = vector(pari.matinverseimage(M,Y).sage())

    # Use the coefficients found to construct the desired polynomials
    
    u = sum([X[j]*L[j] for j in range(c)]) #the polynomial for f 
    v = sum([(-1)^j*X[j]*L[j] for j in range(c)]) #the polynomial for f^
    
    return (u,v)

In [10]:
# Load parameters
nsmall_parameters = load('parameters_ramanujan')

# make a list of bounds
nsmall_bounds = []
for k in range(19):    
    genus = k+2
    systole = nsmall_parameters[k][0]
    param = nsmall_parameters[k][1]
    
    if nev_verify(genus,systole,param) < 2:
        nsmall_bounds += [[genus,systole]]

# return the bounds found    
nsmall_bounds



----------------------------------------------------------------
nev_verify
systole:   2.314252 


Computed the polynomials (u,v) based on the prescribed zeroes
Checked the first sign conditions on f and f^
Verified that f^(r(lambda)) is unimodal on the interval [0,1/4] and determined its minimum
Verified that the roots of f and f^ are exactly as prescribed


----------------------------------------------------------------
nev_verify
systole:   3.21715 


Computed the polynomials (u,v) based on the prescribed zeroes
Checked the first sign conditions on f and f^
Verified that f^(r(lambda)) is unimodal on the interval [0,1/4] and determined its minimum
Verified that the roots of f and f^ are exactly as prescribed


----------------------------------------------------------------
nev_verify
systole:   3.866301 


Computed the polynomials (u,v) based on the prescribed zeroes
Checked the first sign conditions on f and f^
Verified that f^(r(lambda)) is unimodal on the interval [0,1/4] and 

[[2, 2.314252],
 [3, 3.21715],
 [4, 3.866301],
 [5, 4.379112],
 [6, 4.80237],
 [7, 5.167105],
 [8, 5.481318],
 [9, 5.759979],
 [10, 6.009464],
 [11, 6.235534],
 [12, 6.442003],
 [13, 6.63148],
 [14, 6.807348],
 [15, 6.970683],
 [16, 7.123746],
 [17, 7.267285],
 [18, 7.402816],
 [19, 7.530333],
 [20, 7.650766]]